In [4]:
#!pip install openai==0.28
#!pip install python-dotenv

In [5]:
import openai
import os
import pandas as pd
import time
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [6]:
embedding_function = OpenAIEmbeddings()

# Create the language model
llm = ChatOpenAI(model='gpt-4o', temperature=0.0)

analysis_prompt = ChatPromptTemplate.from_messages(
[
    (
    "system",
    """
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{context}"
     """,
     ),
    ("human", "{question}")
    ]
)

In [7]:
def inbody_analysis(embedding_function, llm, prompt, id):
    inbody_dataloader = CSVLoader(file_path='./data/inbody.csv', csv_args={"delimiter": '\n'})
    inbody_data = inbody_dataloader.load()
    #벡터스토어 생성
    inbody_vectorstore = FAISS.from_documents(inbody_data, embedding_function)
    #retriever
    inbody_retriever = inbody_vectorstore.as_retriever()
    
    inbody_chain = (
        {"context": inbody_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    question = f"id가 {id}인 환자에 대해 설명해"
    inbody_response = inbody_chain.invoke(question)
    return inbody_response

In [8]:
def vital_analysis(embedding_function, llm, prompt, id):
    vital_dataloader = CSVLoader(file_path='./data/vital.csv', csv_args={"delimiter": '\n'})
    vital_data = vital_dataloader.load()
    #벡터스토어 생성
    vital_vectorstore = FAISS.from_documents(vital_data, embedding_function)
    #retriever
    vital_retriever = vital_vectorstore.as_retriever()
    
    vital_chain = (
        {"context": vital_retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    question = f"id가 {id}인 환자에 대해 설명해"
    vital_response = vital_chain.invoke(question)
    return vital_response

In [15]:
def final_response(id):
    inbody_response = inbody_analysis(embedding_function, llm, analysis_prompt, id)
    vital_response = vital_analysis(embedding_function, llm, analysis_prompt, id)
    total_analysis = f"""
    inbody 분석 : 
    {inbody_response}
    
    vital 분석 : 
    {vital_response}
    """
    
    final_query = f"""
    You are a professional doctor. Based on the following context, provide a detailed analysis and comprehensive advice regarding the patient's condition. Only include the sections "종합 분석" and "종합적인 조언" in your response. Do not include headings or introductory/concluding sentences. Answer questions using only the provided context. If you do not know the answer, simply state that you do not know; do not speculate or make up information.:\n\n{total_analysis}"""
    
    final_prompt = ChatPromptTemplate.from_messages([
        ("system",
         final_query         
         ),
        ("human", "{question}")
    ])
    final_chain = (
        {"question": RunnablePassthrough()}
        | final_prompt
        | llm
        | StrOutputParser()
    )
    
    question = f"id가 {id}인 환자에 대해 종합적인 건강상태를 분석해"
    response = final_chain.invoke(question)
    return response

In [16]:
print(final_response(0))

종합 분석:
id가 0인 환자의 데이터를 분석한 결과, 2024년 6월 30일과 2024년 7월 17일 두 차례에 걸쳐 측정된 신체 구성 요소를 확인할 수 있습니다. 첫 번째 측정에서는 체지방량이 18.2kg, 총 체수분이 34.7kg, 제지방량이 47.3kg, 골격근량이 25.9kg, 연부조직 제지방량이 44.6kg, BMI가 22.7, 체지방률이 27.7%, 허리-엉덩이 비율이 0.86, 골질량이 2.73kg, 체세포량이 30.5kg, 기초대사량이 1392kcal, 내장지방 레벨이 7로 나타났습니다. 두 번째 측정에서는 체지방량이 17.5kg, 총 체수분이 34.5kg, 제지방량이 47kg, 골격근량이 25.7kg, 연부조직 제지방량이 44.3kg, BMI가 22.4, 체지방률이 27.1%, 허리-엉덩이 비율이 0.86, 골질량이 2.68kg, 체세포량이 30.3kg, 기초대사량이 1386kcal, 내장지방 레벨이 7로 나타났습니다.

환자의 혈압 수치는 전반적으로 정상 범위 내에 있습니다. 수축기 혈압(sbp)은 110에서 124 mmHg 사이, 이완기 혈압(dbp)은 66에서 80 mmHg 사이로 나타났습니다. 이는 일반적으로 건강한 혈압 범위로 간주됩니다. 산소포화도(spo2)는 97%에서 99%로, 정상 범위 내에 있으며, 체온(temp)도 36.2도에서 37도 사이로 정상입니다. 체중(weight)은 64.3kg에서 65.9kg 사이로 약간의 변동이 있지만 큰 변화는 없습니다. 키(height)는 169.7cm에서 170.9cm로 일관되게 나타났습니다. 공복 혈당(glucose)은 101에서 121 mg/dL 사이로, 약간의 변동이 있지만 대체로 정상 범위 내에 있습니다. 맥박(pulse)은 66에서 80 bpm 사이로, 정상 범위 내에 있습니다.

종합적인 조언:
환자의 체지방량과 체지방률이 두 차례 측정에서 모두 약간 감소한 것으로 보입니다. 이는 긍정적인 변화로 볼 수 있으며, 지속적인 관리가 필요합니다. 체수분과 제지방량, 골격근량은 큰 변

In [17]:
print(final_response(1))

종합 분석:
id가 1인 환자의 데이터를 분석한 결과, 2024년 6월 12일부터 2024년 7월 17일까지의 기간 동안 체성분의 변화를 확인할 수 있습니다. 이 기간 동안 환자의 체지방량(body fat mass)은 15.2kg에서 17kg로 증가했다가 다시 16.8kg로 약간 감소했습니다. 총 체수분(total body water)은 28.3kg에서 26.7kg로 감소한 후 26.5kg로 약간 증가했습니다. 제지방량(fat-free mass)과 골격근량(skeletal muscle mass)도 비슷한 패턴을 보이며, 각각 38.6kg에서 36.4kg로, 20.8kg에서 19.5kg로 감소한 후 약간의 변동을 보였습니다. 체질량지수(BMI)는 21.1에서 22.3으로 증가한 후 22.1로 약간 감소했습니다. 체지방률(percent body fat)은 28.2%에서 31.8%로 증가한 후 유지되었습니다. 기초대사량(BMR)은 1203kcal에서 1156kcal로 감소한 후 1149kcal로 약간 더 감소했습니다. 내장지방수치(visceral fat level)는 6에서 7로 증가했습니다.

환자의 혈압, 체온, 체중, 혈당, 맥박 등의 여러 가지 생체 신호를 분석한 결과, 다음과 같은 사항을 확인할 수 있습니다. 환자는 2024년 5월 2일에 항고혈압제를 복용했으며, 이후 기록된 날짜들(2024년 6월 12일, 2024년 7월 3일, 2024년 7월 15일)에는 항고혈압제를 복용하지 않았습니다. 혈압은 전반적으로 안정적인 범위 내에 있으며, 수축기 혈압(SBP)은 112-122 mmHg, 이완기 혈압(DBP)은 59-75 mmHg로 나타났습니다. 산소포화도(SpO2)는 96-98%로 정상 범위에 있습니다. 체온은 35.8-36.6도 사이로 정상 범위에 있으며, 체중은 52.6-54.9kg 사이에서 변동이 있습니다. 키는 155.3-157cm로 기록되어 있습니다. 혈당 수치는 99-122 mg/dL로 나타났으며, 맥박은 64-71 bpm 사이로 안정적입니다.

